# Zoning Sketchbook

In [1]:
# import standard and 3rd-party libraries
import sys
from shapely.geometry import shape
from shapely.ops import transform
import shapely.wkt
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    projections
)

In [2]:
# add udtools core modules to path
module_path = '../../core/python/'
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# import udtools modules
# from geography import wkt
# from geometry.common import coords_to_polygon, fuse_faces, multipolygon_to_faces, polyhedralsurfacez_to_brep, section_at, upgrade_brep, extrude_face
# from geography.coordinate_systems import crs_wgs84, crs_webmercator, crs_nystateplane, wgs84_to_nystateplane, point_nycityhall_wgs84
# from connectors.postgis import connect, prepare_parameterized_query
# from geography.coordinate_systems import test_studyarea_wgs84

from models.zoninglot import ZoningLot
from models.site import Site
from models.scenario import Scenario

In [4]:
# set up an interactive map to pick study area
center = [40.809617674105, -73.908714998524]
zoom = 16

bounds = None
bounds_nysp = None
bounds_center = None
bounds_center_nysp = None

m = Map(crs=projections.EPSG3857, center=center, zoom=zoom)

dc = DrawControl(
    marker={'shapeOptions': {'color': '#0000FF'}},
    rectangle={'shapeOptions': {'color': '#0000FF'}},
    circle={'shapeOptions': {'color': '#0000FF'}},
    circlemarker={},
)

def set_bounds(b):
    global bounds, bounds_nysp, bounds_center, bounds_center_nysp
    bounds = b
    bounds_center = bounds.representative_point()
    bounds_nysp = transform(wgs84_to_nystateplane, bounds)
    bounds_center_nysp = transform(wgs84_to_nystateplane, bounds_center)
    print(bounds_nysp)

def handle_draw(target, action, geo_json):
    bounds = shape(dc.last_draw['geometry'])
    print(bounds.wkt)
    set_bounds(bounds)

dc.on_draw(handle_draw)
m.add_control(dc)

In [5]:
# show the map
m

Map(center=[40.809617674105, -73.908714998524], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [6]:
# get query bounds and area centroid from default
aoi = shapely.wkt.loads('POLYGON ((1009405.90154069 234515.39897036, 1009930.58999844 234367.63467916, 1009849.1944143 234052.07026066, 1009314.48803851 234194.82559284, 1009405.90154069 234515.39897036))')

# define common query params for study area
params = {
    'center_x': aoi.representative_point().x,
    'center_y': aoi.representative_point().y,
    'bounds': aoi.wkt
}
params

# or from interactive map (uncomment)
# set_bounds(shape(dc.last_draw['geometry']))

{'center_x': 1009623.7149731631,
 'center_y': 234281.230136,
 'bounds': 'POLYGON ((1009405.90154069 234515.39897036, 1009930.58999844 234367.63467916, 1009849.1944143 234052.07026066, 1009314.48803851 234194.82559284, 1009405.90154069 234515.39897036))'}

## Get models and do things with them

In [7]:
# define site, scenario, zoning lot
my_scenario = Scenario('A')
my_site = my_site = Site('1', ['2025770020', '2025770022'])
zl = ZoningLot(my_scenario, my_site)

(OLD)

In [ ]:
# create an envelope
massing_goals = [
    {'height': 15.0, 'usegroup': 4, 'label': 'school'},
    {'height': 10.0, 'usegroup': 2, 'label': 'residential'},
]

envelope = zl.generate_envelope(massing_goals)

In [ ]:
# create a proposed building using the default massing goals
massing = zl.generate_building(massing_goals)

In [ ]:
# summarize
# study.summarize() # summarize the whole study, including increment
my_scenario.summarize() # summarize everything in the current scenario
massing.summarize() # summarize the current proposed building

## Preview the results

In [ ]:
# set up renderer
renderer = JupyterRenderer()

In [ ]:
# display results
# envelope
# massing
# fuse_result.Shape()
# bldgs[6].geom
# zl.geom.Shape()
renderer.DisplayShape(envelope, render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

In [ ]:
for floor in massing.floors:
    renderer.DisplayShape(floor, render_edges=True)

## Export to STEP file

(can be opened with Rhino)

In [ ]:
# from connectors.step import shape_to_step
# shape_to_step(envelope.Shape(), "/Users/carsten/Desktop/test.stp")